# Boiler

In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from glob import glob

import dask
from dask.diagnostics import ProgressBar
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
# import dask.distributed as dd
# import tempfile

# tempdir = tempfile.TemporaryDirectory("dask-worker-space")
# dd.Client(local_directory=tempdir.name, memory_limit='16gb')

# from distributed import Client
# client = Client()

from dask.distributed import Client
client = Client()
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42547,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://127.0.0.1:38917,Total threads: 4
Dashboard: /proxy/41161/status,Memory: 11.23 GiB
Nanny: tcp://127.0.0.1:36001,


# RMM Data

In [3]:
MJO_DIR = '/g/data/w40/ab2313/rainfall_2020/mjo_index'


In [14]:
rmm_raw_ds = xr.open_dataset(os.path.join(MJO_DIR, 'mjo_daily_clim_1981_2010_1979-2022.nc'))
rmm1_raw_ds = rmm_raw_ds.amplitude * np.cos(rmm_raw_ds.phase); rmm2_raw_ds = rmm_raw_ds.amplitude * np.sin(rmm_raw_ds.phase)
rmm_ds = xr.merge([rmm1_raw_ds.to_dataset(name='rmm1'), rmm2_raw_ds.to_dataset(name='rmm2'), rmm_raw_ds[['amplitude']]])
rmm_ds

<xarray.Dataset>
Dimensions:    (time: 16040)
Coordinates:
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 327 328 329 330 331 332 333 334
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-11-30
Data variables:
    rmm1       (time) float64 2.859e+08 3.517e+08 ... 4.547e+07 -8.453e+07
    rmm2       (time) float64 -4.515e+08 -2.58e+08 ... 2.1e+08 2.273e+08
    amplitude  (time) float64 ...

In [15]:
bound = float(rmm_ds.amplitude.mean() - rmm_ds.amplitude.std().values)
bound

217801725.39620945

# AGCD Data

In [ ]:
AGCD_DIR = '/g/data/zv2/agcd/v1/precip/calib/r005/01day'

In [ ]:
files = glob(os.path.join(AGCD_DIR, '*.nc'))
files = np.sort([f for f in files if int(f.split('_')[-1].split('.')[0]) >= 1974]) # Just files with year >1974
files[:5]

In [5]:
# 3https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6475_9317_5747_6204
agcd_ds = xr.open_mfdataset(files)
agcd_ds = agcd_ds.precip.chunk({'time':-1, 'lat':int(691/20), 'lon':(int(886/16))}) # Chunking in time
agcd_ds['time'] = agcd_ds.time.values - pd.to_timedelta('9H') # Adjust time to match rmm
agcd_ds = agcd_ds.where(agcd_ds > 1) # Remove rainfall less than 1mm
agcd_ds

,Array,Chunk
Bytes,38.67 GiB,120.93 MiB
Shape,"(16953, 691, 886)","(16953, 34, 55)"
Count,3908 Tasks,357 Chunks
Type,float32,numpy.ndarray


In [26]:
agcd_monthly_sum_climatology_ds = agcd_ds.groupby('time.season').sum(dim='time')
agcd_monthly_count_climatology_ds = agcd_ds.groupby('time.season').count(dim='time')
agcd_rprd_clim = agcd_monthly_sum_climatology_ds/agcd_monthly_count_climatology_ds

In [ ]:
agcd_rprd_clim

In [28]:
with ProgressBar():
    agcd_rprd_clim = agcd_rprd_clim.compute()

2023-04-28 13:52:42,210 - distributed.worker_memory - WARNING - gc.collect() took 3.141s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2023-04-28 13:52:49,811 - distributed.worker_memory - WARNING - gc.collect() took 3.072s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2023-04-28 13:52:55,314 - distributed.worker_memory - WARNING - gc.collect() took 2.567s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2023-04-28 13:53:04,658 - distributed.worker_memory - WARNING - gc.collect() took 2.053s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2023-04-28 13:54:12,316 - distributed.worker_memory - WARNING - Worker exceeded 95% 

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/distributed/client.py", line 3018, in get
    results = self.gather(packed, asynchronous=asynchronous, direct=direct)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/distributed/client.py", line 2171, in gather
    return self.sync(
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/distributed/utils.py", line 309, in sync
    return sync(
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/distributed/utils.py", line 372, in sync
    wait(10)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/distributed/utils.py", line 361, in wait
    return e.wait(timeout)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/threading.py", line 574, in wait
    signaled = self._cond.wai


KeyboardInterrupt



In [ ]:
def split_da_into_rmm_phases(da, rmm_da, bound=1):
    
    rmm_inact_dates = rmm_da.where(rmm_da.amplitude < bound, drop = True).time.values
    inact_da = da.where(da.time.isin(rmm_inact_dates), drop = True)
    
    single_phase = [] 
    rmm_act = rmm_da.where(rmm_da.amplitude >= bound, drop = True)
    phases = np.arange(1,9) 
    for phase in phases:
        rmm_single_dates = rmm_act.where(rmm_act.phase_ID == phase, drop = True).time.values
        datafile_single = da.where(da.time.isin(rmm_single_dates), drop = True)
        single_phase.append(datafile_single) 

    phases = np.append(phases, 0)
    
    # The inactive phase also needs to be included
    single_phase.append(inact_da) 

    datafile_RMM_split = xr.concat(single_phase, pd.Index(phases, name = 'phase_ID'))

    return  datafile_RMM_split

In [ ]:
agcd_rmm_phase_ds = split_da_into_rmm_phases(agcd_ds.chunk('auto'), rmm_raw_ds.chunk('auto'), bound=bound)
agcd_rmm_phase_ds = agcd_rmm_phase_ds.chunk('auto')

In [ ]:
agcd_rmm_sum = agcd_rmm_phase_ds.groupby('time.month').sum()
agcd_rmm_count = agcd_rmm_phase_ds.groupby('time.month').count()

In [ ]:
agcd_rmm_sum

In [ ]:
agcd_rmm_sum.isel(phase=0, month=1).plot()

# Phase Grouping

In [ ]:
rmm_raw_ds_act = rmm_raw_ds.where(rmm_raw_ds.amplitude > bound, drop=True)

In [ ]:
grouped_stor = []; phases = []
for group in rmm_raw_ds_act.amplitude.groupby(rmm_raw_ds_act.phase_ID):
    single_phase_count = group[1].groupby('time.month').count()
    grouped_stor.append(single_phase_count); phases.append(group[0])

In [ ]:
rmm_act_phase_month_count = xr.concat(grouped_stor, pd.Index(np.array(phases).astype(int), name='phase_ID'))
rmm_act_phase_month_count = rmm_act_phase_month_count.to_dataset(name='number')
rmm_act_phase_month_count

In [ ]:
rmm_raw_ds_inact = rmm_raw_ds.where(rmm_raw_ds.amplitude < bound, drop=True)
rmm_inact_count = rmm_raw_ds_inact.amplitude.groupby('time.month').count().to_dataset(name='number')
rmm_inact_count = rmm_inact_count.assign_coords(phase_ID=("phase_ID", [0]))
rmm_inact_count

In [ ]:
rmm_phase_month_count = rmm_act_phase_month_count.combine_first(rmm_inact_count)
rmm_phase_month_count

# Anomalies

In [ ]:
agcd_rmm_rpd = agcd_rmm_sum/rmm_phase_month_count

In [ ]:
agcd_rmm_rprd = agcd_rmm_sum/agcd_rmm_count
agcd_rmm_rprd

In [ ]:
with ProgressBar():
    agcd_rmm_rprd = agcd_rmm_rprd.chunk('auto').persist(method='proccess')